In [1]:
#FL reference: https://towardsdatascience.com/preserving-data-privacy-in-deep-learning-part-1-a04894f78029

In [2]:
import medmnist
from medmnist import INFO, Evaluator

import os
import random
from tqdm import tqdm
import numpy as np
import torch, torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data.dataset import Dataset   
torch.backends.cudnn.benchmark=True



In [3]:
#pip install medmnist

In [4]:
from tqdm import tqdm
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torchvision.transforms as transforms



In [5]:
##### Hyperparameters for federated learning #########
num_clients = 3
num_selected = 3
num_rounds = 2
epochs = 3
batch_size = 128

In [6]:
print(f"MedMNIST v{medmnist.__version__} @ {medmnist.HOMEPAGE}")

MedMNIST v2.1.0 @ https://github.com/MedMNIST/MedMNIST/


In [7]:
data_flag = 'chestmnist'
download = True

NUM_EPOCHS = 3
BATCH_SIZE = 128
lr = 0.001

info = INFO[data_flag]
task = info['task']
n_channels = info['n_channels']
n_classes = len(info['label'])

DataClass = getattr(medmnist, info['python_class'])

In [8]:
# preprocessing
data_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[.5], std=[.5])
])

# load the data
train_dataset = DataClass(split='train', transform=data_transform, download=download)
test_dataset = DataClass(split='test', transform=data_transform, download=download)

traindata_split = torch.utils.data.random_split(train_dataset, [int(train_dataset.imgs.shape[0] / num_clients) for _ in range(num_clients)])

train_loader = [torch.utils.data.DataLoader(x, batch_size=batch_size, shuffle=True) for x in traindata_split]

#test_loader = [torch.utils.data.DataLoader(x, batch_size=batch_size, shuffle=True) for x in testdata_split]



# # Dividing the training data into num_clients, with each client having equal number of images
# traindata_split = torch.utils.data.random_split(train_dataset, [int(np.shape(train_dataset)[0] / num_clients) for _ in range(num_clients)])
# # Creating a pytorch loader for a Deep Learning model
# train_loader = [torch.utils.data.DataLoader(x, batch_size=batch_size, shuffle=True) for x in traindata_split]
# # Loading the test iamges and thus converting them into a test_loader
# 
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)


# pil_dataset = DataClass(split='train', download=download)

# encapsulate data into dataloader form
#train_loader = data.DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)

#test_loader = data.DataLoader(dataset=test_dataset, batch_size=2*BATCH_SIZE, shuffle=False)

Using downloaded and verified file: C:\Users\ibrah\.medmnist\chestmnist.npz
Using downloaded and verified file: C:\Users\ibrah\.medmnist\chestmnist.npz


In [9]:
test_dataset.imgs.shape[0]

22433

In [10]:
print(train_dataset)
print("===================")
print(test_dataset)

Dataset ChestMNIST (chestmnist)
    Number of datapoints: 78468
    Root location: C:\Users\ibrah\.medmnist
    Split: train
    Task: multi-label, binary-class
    Number of channels: 1
    Meaning of labels: {'0': 'atelectasis', '1': 'cardiomegaly', '2': 'effusion', '3': 'infiltration', '4': 'mass', '5': 'nodule', '6': 'pneumonia', '7': 'pneumothorax', '8': 'consolidation', '9': 'edema', '10': 'emphysema', '11': 'fibrosis', '12': 'pleural', '13': 'hernia'}
    Number of samples: {'train': 78468, 'val': 11219, 'test': 22433}
    Description: The ChestMNIST is based on the NIH-ChestXray14 dataset, a dataset comprising 112,120 frontal-view X-Ray images of 30,805 unique patients with the text-mined 14 disease labels, which could be formulized as a multi-label binary-class classification task. We use the official data split, and resize the source images of 1×1024×1024 into 1×28×28.
    License: CC BY 4.0
Dataset ChestMNIST (chestmnist)
    Number of datapoints: 22433
    Root location: C:

In [11]:
# visualization
train_dataset.montage(length=1)

C:\ProgramData\Anaconda3\lib\site-packages\medmnist\utils.py:25: FutureWarning: `multichannel` is a deprecated argument name for `montage`. It will be removed in version 1.0. Please use `channel_axis` instead.
  montage_arr = skimage_montage(sel_img, multichannel=(n_channels == 3))


In [12]:
# define a simple CNN model

class Net(nn.Module):
    def __init__(self, in_channels, num_classes):
        super(Net, self).__init__()

        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels, 16, kernel_size=3),
            nn.BatchNorm2d(16),
            nn.ReLU())

        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 16, kernel_size=3),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))

        self.layer3 = nn.Sequential(
            nn.Conv2d(16, 64, kernel_size=3),
            nn.BatchNorm2d(64),
            nn.ReLU())
        
        self.layer4 = nn.Sequential(
            nn.Conv2d(64, 64, kernel_size=3),
            nn.BatchNorm2d(64),
            nn.ReLU())

        self.layer5 = nn.Sequential(
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))

        self.fc = nn.Sequential(
            nn.Linear(64 * 4 * 4, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, num_classes))

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.layer5(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

model = Net(in_channels=n_channels, num_classes=n_classes)
    
# define loss function and optimizer
if task == "multi-label, binary-class":
    criterion = nn.BCEWithLogitsLoss()
else:
    criterion = nn.CrossEntropyLoss()
    
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)

In [13]:
def client_update(client_model, optimizer, train_loader, epoch=5):
    """
    This function updates/trains client model on client data
    """
    model.train()
    for e in range(epoch):
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.cuda(), target.cuda().float()
            optimizer.zero_grad()
            output = client_model(data).float()
            loss = F.cross_entropy(output, target)
            loss.backward()
            optimizer.step()
    return loss.item()

In [14]:

def server_aggregate(global_model, client_models):
    """
    This function has aggregation method 'mean'
    """
    ### This will take simple mean of the weights of models ###
    global_dict = global_model.state_dict()
    for k in global_dict.keys():
        global_dict[k] = torch.stack([client_models[i].state_dict()[k].float() for i in range(len(client_models))], 0).mean(0)
    global_model.load_state_dict(global_dict)
    for model in client_models:
        model.load_state_dict(global_model.state_dict())

In [15]:
task = info['task']


In [16]:
from medmnist import INFO, Evaluator
def test(global_model, test_loader):
    model.eval()
    y_true = torch.tensor([]).cuda().float()
    y_score = torch.tensor([]).cuda().float()
    
    data_loader = test_loader

    with torch.no_grad():
        for inputs, targets in data_loader:
            outputs = global_model(inputs.cuda().float())

            if task == 'multi-label, binary-class':
                targets = targets.cuda().float()
                outputs = outputs.softmax(dim=-1).cuda().float()
            else:
                targets = targets.squeeze().cuda().float()
                outputs = outputs.softmax(dim=-1).cuda().float()
                targets = targets.float().resize_(len(targets), 1).cuda().float()

            y_true = torch.cat((y_true, targets), 0)
            y_score = torch.cat((y_score, outputs), 0)

        y_true = y_true.cpu().numpy()

        y_score = y_score.cpu().numpy()
        
        evaluator = Evaluator(data_flag, 'test')
        metrics = evaluator.evaluate(y_score)
    
        print('%s  auc: %.3f  acc:%.3f' % ('test', *metrics))
        return metrics.ACC, metrics.AUC

In [17]:

############################################
#### Initializing models and optimizer  ####
############################################

#### global model ##########
global_model =  Net(in_channels=n_channels, num_classes=n_classes).cuda()

############## client models ##############
client_models = [ Net(in_channels=n_channels, num_classes=n_classes).cuda() for _ in range(num_selected)]
for model in client_models:
    model.load_state_dict(global_model.state_dict()) ### initial synchronizing with global model 

############### optimizers ################
opt = [optim.SGD(model.parameters(), lr=0.1) for model in client_models]

In [18]:
# # train

# for epoch in range(NUM_EPOCHS):
#     train_correct = 0
#     train_total = 0
#     test_correct = 0
#     test_total = 0
    
#     model.train()
#     for inputs, targets in tqdm(train_loader):
#         # forward + backward + optimize
#         inputs, targets = inputs.cuda(), targets.cuda() # add this line

#         optimizer.zero_grad()
#         outputs = model(inputs)
        
#         if task == 'multi-label, binary-class':
#             targets = targets.to(torch.float32)
#             loss = criterion(outputs, targets)
#         else:
#             targets = targets.squeeze().long()
#             loss = criterion(outputs, targets)
        
#         loss.backward()
#         optimizer.step()

In [19]:
###### List containing info about learning #########
losses_train = []
auc_test = []
acc_train = []
acc_test = []
# Runnining FL

for r in range(num_rounds):
    # select random clients
    client_idx = np.random.permutation(num_clients)[:num_selected]
    # client update
    loss = 0
    for i in tqdm(range(num_selected)):
        loss += client_update(client_models[i], opt[i], train_loader[client_idx[i]], epoch=epochs)
    
    losses_train.append(loss)
    # server aggregate
    server_aggregate(global_model, client_models)
    
    acc, auc = test(global_model, test_loader)
    auc_test.append(auc)
    acc_test.append(acc)
    print('%d-th round' % r)
    print('average train loss %0.3g | auc test %0.3g | test acc: %0.3f' % (loss / num_selected, auc, acc))


100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [01:23<00:00, 27.96s/it]


test  auc: 0.504  acc:0.947
0-th round
average train loss 1.7 | auc test 0.504 | test acc: 0.947


100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [01:11<00:00, 23.93s/it]


test  auc: 0.497  acc:0.947
1-th round
average train loss 1.35 | auc test 0.497 | test acc: 0.947


In [ ]:
print(torch.cuda.is_available())
torch.cuda.device_count()
torch.cuda.current_device()
torch.cuda.device(0)
torch.cuda.get_device_name(0)